In [158]:
import os
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import re
from collections import Counter

In [146]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import LinearRegression

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [216]:
STATS_DIR = "/hg191/corpora/academic-data/semantic-scholar/stats/"
CURRENT_YEAR = 2018
N=50000

In [10]:
! ls $STATS_DIR

abs.2M.early.docs  abs.docs  abs.ind	   abs.npages	abs.nuniqs  abs.years
abs.2M.later.docs  abs.ids   abs.nauthors  abs.ntokens	abs.outd


In [7]:
with open (os.path.join (STATS_DIR, "abs.ids")) as fin:
    ids = {line.strip(): i for i, line in enumerate (fin)}
iids = {value: key for key, value in ids.items()}

In [38]:
indices = np.random.choice(len(ids), N, replace=False)

In [136]:
def partsNotNumeric (parts):
    return any([any([c.isalpha() for c in part]) for part in parts])

In [137]:
def doBestNonNumericExtraction (parts):
    parts = [''.join(c for c in part if c.isdigit()) for part in parts]
    return doBestNumericExtraction (parts)

In [138]:
def doBestNumericExtraction (parts): 
    if len (parts[0]) <= len (parts[1]) and ":" not in parts[0]:
        return int(float(parts[1]) - float (parts[0]) + 1)
    elif ":" in parts[0]:
        return int(float(parts[1].split(":")[1]) - float (parts[0].split(":")[1]) + 1)
    else: 
        return int (float (parts[0][0:len(parts[0])-len (parts[1])]+parts[1]) - float (parts[0]) + 1)

In [139]:
def npages_fromstring(string):
    DEFAULT_PAGES = 1
    try:
        if len(string.strip()) == 0:
            return DEFAULT_PAGES 
        # remove extraneous information
        if "," in string:
            string = string.strip().split(",")[0]
        if ";" in string:
            string = string.strip().split(";")[0]

        # if only one page number is provided
        if len(string.strip().split("-")) == 1:
            return DEFAULT_PAGES
        # if range is specified
        elif len (string.strip().split("-")) == 2:
            parts = list (map(lambda x:x.strip(), string.strip().split ("-")))
            # end limit of range not specified
            if len(parts[1]) == 0:
                return DEFAULT_PAGES
            #parts = [''.join(c for c in part if c.isdigit()) for part in parts]
            if partsNotNumeric (parts):
                return doBestNonNumericExtraction (parts)
            else:
                return doBestNumericExtraction (parts)                
        else:
            return DEFAULT_PAGES
    except Exception:
        #print ("Exception", string)
        return DEFAULT_PAGES

In [11]:
# read the outdegrees feature
with open (os.path.join (STATS_DIR, "abs.outd")) as fin:
    outd = dict ()
    for line in fin:
        parts = line.strip().split (",")
        outd[ids[parts[0]]] = int(parts[1])
        
# read the years feature
with open (os.path.join (STATS_DIR, "abs.years")) as fin:
    years = dict ()
    for line in fin:
        parts = line.strip().split(",")
        years[ids[parts[0]]] = int (parts[1])

In [13]:
# read the abstract length feature
with open (os.path.join (STATS_DIR, "abs.nuniqs")) as fin:
    nuniqs = dict ()
    for line in fin:
        parts = line.strip().split (",")
        nuniqs[ids[parts[0]]] = int (parts[1])

# read the number of authors feature
with open (os.path.join (STATS_DIR, "abs.nauthors")) as fin:
    nauthors = dict ()
    for line in fin:
        parts = line.strip().split(",")
        nauthors[ids[parts[0]]] = int (parts[1])

# read the indegree dependent variable
with open (os.path.join (STATS_DIR, "abs.ind")) as fin:
    ind = dict ()
    for line in fin:
        parts = line.strip().split(",")
        ind[ids[parts[0]]] = int (parts[1])

In [140]:
def my_clip(num, min_val, max_val):
    if num < min_val:
        return min_val
    elif num > max_val:
        return max_val
    else:
        return num

In [142]:
# read the document length feature
with open (os.path.join (STATS_DIR, "abs.npages")) as fin:
    npages = dict ()
    for i, line in enumerate(fin):
        parts = line.strip().split(",")
        pages_estimate = npages_fromstring(parts[1])
        npages[ids[parts[0]]] = my_clip(pages_estimate, 1, 50)
        if i % 1000000 == 0:
            print ("Lines processed: {0}".format (i))

Lines processed: 0
Lines processed: 1000000
Lines processed: 2000000
Lines processed: 3000000
Lines processed: 4000000
Lines processed: 5000000
Lines processed: 6000000
Lines processed: 7000000
Lines processed: 8000000
Lines processed: 9000000
Lines processed: 10000000
Lines processed: 11000000
Lines processed: 12000000
Lines processed: 13000000
Lines processed: 14000000
Lines processed: 15000000
Lines processed: 16000000
Lines processed: 17000000
Lines processed: 18000000
Lines processed: 19000000
Lines processed: 20000000
Lines processed: 21000000
Lines processed: 22000000
Lines processed: 23000000
Lines processed: 24000000
Lines processed: 25000000
Lines processed: 26000000
Lines processed: 27000000


**1. Dependent variable**

In [163]:
y = np.array ([ind[index] for index in indices])

**2. Independent variables**

(a) Outdegree model

In [40]:
X_outd = np.array ([outd[index] for index in indices])
X = sm.add_constant (X_outd)

In [41]:
#results = sm.OLS(y, X).fit()
results = sm.GLM(y,X,family=sm.families.NegativeBinomial()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                50000
Model:                            GLM   Df Residuals:                    49998
Model Family:        NegativeBinomial   Df Model:                            1
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5781e+05
Date:                Tue, 20 Nov 2018   Deviance:                   1.3736e+05
Time:                        21:07:01   Pearson chi2:                 1.39e+06
No. Iterations:                    13   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9522      0.005    370.340      0.000       1.942       1.963
x1             0.0165      0.000     65.288      0.0

(b) +age

In [42]:
X_age = np.array ([CURRENT_YEAR - years[index] for index in indices])
X = np.array([X_outd, X_age]).T
X = sm.add_constant(X)

In [43]:
results = sm.GLM(y,X,family=sm.families.NegativeBinomial()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                50000
Model:                            GLM   Df Residuals:                    49997
Model Family:        NegativeBinomial   Df Model:                            2
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5495e+05
Date:                Tue, 20 Nov 2018   Deviance:                   1.3166e+05
Time:                        21:07:14   Pearson chi2:                 1.18e+06
No. Iterations:                    17   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2498      0.008    147.773      0.000       1.233       1.266
x1             0.0223      0.000     85.654      0.0

(c) + Age^2

In [44]:
X_age2 = np.array ([(CURRENT_YEAR - years[index]) ** 2 for index in indices])
X = np.array([X_outd, X_age, X_age2]).T
X = sm.add_constant(X)

In [45]:
results = sm.GLM(y,X,family=sm.families.NegativeBinomial()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                50000
Model:                            GLM   Df Residuals:                    49996
Model Family:        NegativeBinomial   Df Model:                            3
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4959e+05
Date:                Tue, 20 Nov 2018   Deviance:                   1.2094e+05
Time:                        21:07:35   Pearson chi2:                 1.37e+06
No. Iterations:                    42   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3291      0.012     26.945      0.000       0.305       0.353
x1             0.0248      0.000     93.789      0.0

(d) abstract length

In [46]:
X_nuniqs = np.array ([nuniqs[index] for index in indices])
X = np.array([X_outd, X_age, X_age2, X_nuniqs]).T
X = sm.add_constant(X)

In [47]:
results = sm.GLM(y,X,family=sm.families.NegativeBinomial()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                50000
Model:                            GLM   Df Residuals:                    49995
Model Family:        NegativeBinomial   Df Model:                            4
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4903e+05
Date:                Tue, 20 Nov 2018   Deviance:                   1.1980e+05
Time:                        21:07:56   Pearson chi2:                 1.30e+06
No. Iterations:                    45   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1672      0.016    -10.397      0.000      -0.199      -0.136
x1             0.0242      0.000     91.379      0.0

(e) nauthors

In [48]:
X_nauthors = np.array ([nauthors[index] for index in indices])
X = np.array([X_outd, X_age, X_age2, X_nuniqs, X_nauthors]).T
X = sm.add_constant(X)

In [49]:
results = sm.GLM(y,X,family=sm.families.NegativeBinomial()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                50000
Model:                            GLM   Df Residuals:                    49994
Model Family:        NegativeBinomial   Df Model:                            5
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4844e+05
Date:                Tue, 20 Nov 2018   Deviance:                   1.1863e+05
Time:                        21:08:11   Pearson chi2:                 1.48e+06
No. Iterations:                    48   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.3491      0.017    -20.954      0.000      -0.382      -0.316
x1             0.0235      0.000     88.278      0.0

(f) npages

In [160]:
X_npages = np.array ([npages[index] for index in indices])
X = np.array ([X_outd, X_age, X_age2, X_nuniqs, X_nauthors, X_npages]).T
X = sm.add_constant (X)

In [211]:
results = sm.GLM(y,X,family=sm.families.NegativeBinomial()).fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                50000
Model:                            GLM   Df Residuals:                    49993
Model Family:        NegativeBinomial   Df Model:                            6
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4641e+05
Date:                Wed, 21 Nov 2018   Deviance:                   1.1458e+05
Time:                        18:13:14   Pearson chi2:                 1.46e+06
No. Iterations:                    47   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6757      0.017    -38.641      0.000      -0.710      -0.641
x1             0.0208      0.000     77.168      0.0

In [164]:
po_results = sm.GLM(y, X, family=sm.families.Poisson()).fit()
print(po_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                50000
Model:                            GLM   Df Residuals:                    49993
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.5817e+05
Date:                Wed, 21 Nov 2018   Deviance:                   1.2032e+06
Time:                        17:36:06   Pearson chi2:                 1.93e+07
No. Iterations:                    15   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1989      0.006     34.637      0.000       0.188       0.210
x1             0.0106   3.73e-05    284.981      0.0

In [187]:
def estimate_overdispersion (orig_x, orig_y):
    "Calculates response observation for Cameron-Trivedi dispersion test"
    def ct_response (xx, yy):
        return np.array([((yy[i] - xx[i])**2 - yy[i]) / xx[i] for i in range (len (xx))])
    ct_results = sm.OLS (ct_response(orig_x, orig_y), orig_x).fit()
    alpha_ci95 = ct_results.conf_int(0.05)
    print('\nC-T dispersion test: alpha = {:5.3f}, 95% CI = ({:5.3f}, {:5.3f})'
        .format(ct_results.params[0], alpha_ci95[0,0], alpha_ci95[0,1]))
    return ct_results.params[0]

In [188]:
overdispersion = estimate_overdispersion (po_results.mu, y)


C-T dispersion test: alpha = 10.153, 95% CI = (-12.095, 32.402)


In [192]:
results = sm.GLM(y,X,family=sm.families.NegativeBinomial(alpha=overdispersion)).fit(maxiter=500)
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                50000
Model:                            GLM   Df Residuals:                    49993
Model Family:        NegativeBinomial   Df Model:                            6
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4593e+05
Date:                Wed, 21 Nov 2018   Deviance:                       19446.
Time:                        18:00:01   Pearson chi2:                 1.71e+05
No. Iterations:                   500   Covariance Type:             nonrobust
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7588      0.049    -15.636      0.000      -0.854      -0.664
x1             0.0220      0.001     26.616      0.0

In [208]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error, explained_variance_score, r2_score

In [212]:
# Poission model
print (mean_squared_error (y, po_results.fittedvalues))
print (mean_squared_log_error (y, po_results.fittedvalues))
print (explained_variance_score(y, po_results.fittedvalues))
print (r2_score(y, po_results.fittedvalues))

2139.2139644433223
2.0137926140715035
-0.02300044269248902
-0.0230004426924888


In [214]:
# Negative binomial
print (mean_squared_error (y, results.fittedvalues))
print (mean_squared_log_error (y, results.fittedvalues))
print (explained_variance_score(y, results.fittedvalues))
print (r2_score(y, results.fittedvalues))

4.650473886977446e+16
1.9908983985650868
-22237863070774.473
-22239181887285.31


In [213]:
# Negative binomial (after overdispersion correction)
print (mean_squared_error (y, results.fittedvalues))
print (mean_squared_log_error (y, results.fittedvalues))
print (explained_variance_score(y, results.fittedvalues))
print (r2_score(y, results.fittedvalues))

4.650473886977446e+16
1.9908983985650868
-22237863070774.473
-22239181887285.31


(g) bow

Fit an ordinary least squares model with bag of words as features to predict citation count.